<a href="https://colab.research.google.com/github/Hanbin-git/Dacon_cacer/blob/main/2025_06_19(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip -o "/content/drive/MyDrive/open_1.zip" -d "/content/open_1"


Archive:  /content/drive/MyDrive/open_1.zip
  inflating: /content/open_1/sample_submission.csv  
  inflating: /content/open_1/test.csv  
  inflating: /content/open_1/train.csv  


In [3]:
import os

def get_path(filename):
    return "/content/open_1/" + filename


In [4]:
# SMOTE 설치
!pip install -U imbalanced-learn
!pip install lightgbm optuna
!pip install -q lightgbm catboost xgboost



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.2 MB/s eta 0:00:00


In [5]:
!pip install --upgrade lightgbm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0


In [6]:
import lightgbm as lgb
print(lgb.__version__)

4.6.0


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 함수
def get_path(filename):
    return "/content/open_1/" + filename  # 환경에 맞게 수정

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ 라벨, 특징 분리
y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 (인코딩 + 결측치)
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 기본 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)

# ✅ 메타 모델 (LGBM)
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

# ✅ 스태킹 구성
base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 전체 학습 후 예측
stack_model.fit(X, y)
test_preds = stack_model.predict_proba(X_test)[:, 1]
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_lgb_meta.csv", index=False)


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004788 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002388 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003178 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Threshold: 0.26, Best F1 Score: 0.48624


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002985 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [8]:
from google.colab import files
files.download("submission_stacking_lgb_meta.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 설정 함수
def get_path(filename):
    return "/content/open_1/" + filename  # 경로는 환경에 따라 조정

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

# ✅ 라벨, 피처 분리
y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test_base, _ = preprocess(X_test, encoders)

# ✅ 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)

meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

stack_model = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    stack_method="predict_proba",
    n_jobs=-1
)

# ✅ Threshold 최적화
oof_preds = cross_val_predict(stack_model, X, y, method="predict_proba", cv=5)[:, 1]
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 전체 학습
stack_model.fit(X, y)

# ✅ TTA 적용: Gaussian Noise 기반 예측 평균
def apply_tta(model, X_test, n_iter=5, noise_std=0.01):
    preds = []
    for _ in range(n_iter):
        X_noised = X_test.copy()
        numeric_cols = X_noised.select_dtypes(include=[np.number]).columns
        X_noised[numeric_cols] += np.random.normal(loc=0.0, scale=noise_std, size=X_noised[numeric_cols].shape)
        preds.append(model.predict_proba(X_noised)[:, 1])
    return np.mean(preds, axis=0)

tta_preds = apply_tta(stack_model, X_test_base, n_iter=5, noise_std=0.01)

# ✅ 최종 예측 및 저장
submission["Cancer"] = (tta_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_lgb_meta_TTA.csv", index=False)


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003760 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002347 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003749 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 8368, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003951 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69728, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.120009 -> initscore=-1.992343
[LightGBM] [Info] Start training from score -1.992343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Best Threshold: 0.26, Best F1 Score: 0.48624
[LightGBM] [Info] Number of positive: 10459, number of negative: 76700
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002973 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 87159, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119999 -> initscore=-1.992439
[LightGBM] [Info] Start training from score -1.992439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [10]:
from google.colab import files
files.download("submission_stacking_lgb_meta_TTA.csv")
# 점수 떨어짐

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# ✅ 경로 설정
def get_path(filename):
    return "/content/open_1/" + filename

# ✅ 데이터 로딩
train = pd.read_csv(get_path("train.csv"))
test = pd.read_csv(get_path("test.csv"))
submission = pd.read_csv(get_path("sample_submission.csv"))

y = train["Cancer"]
X = train.drop(columns=["ID", "Cancer"])
X_test = test.drop(columns=["ID"])

# ✅ 전처리 함수
def preprocess(df, encoders=None):
    df = df.copy()
    encoders = encoders or {}
    for col in df.columns:
        if df[col].dtype == "object":
            df[col] = df[col].astype(str)
            if col not in encoders:
                encoders[col] = LabelEncoder().fit(df[col])
            df[col] = encoders[col].transform(df[col])
        else:
            df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]])
    return df, encoders

X, encoders = preprocess(X)
X_test, _ = preprocess(X_test, encoders)

# ✅ 모델 정의
model_lgbm = LGBMClassifier(learning_rate=0.05, max_depth=6, num_leaves=31, random_state=42)
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_cat = CatBoostClassifier(verbose=0, random_state=42)
meta_model = LGBMClassifier(learning_rate=0.05, max_depth=3, num_leaves=15, random_state=42)

base_models = [
    ('lgbm', model_lgbm),
    ('xgb', model_xgb),
    ('cat', model_cat)
]

# ✅ 앙상블 예측 저장용
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
test_preds = np.zeros(X_test.shape[0])
oof_preds = np.zeros(X.shape[0])

# ✅ Fold 별 학습 및 예측
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

    stack_model = StackingClassifier(
        estimators=base_models,
        final_estimator=meta_model,
        cv=5,
        stack_method="predict_proba",
        n_jobs=-1
    )

    stack_model.fit(X_train, y_train)

    oof_preds[val_idx] = stack_model.predict_proba(X_val)[:, 1]
    test_preds += stack_model.predict_proba(X_test)[:, 1] / kf.n_splits
    print(f"Fold {fold + 1} F1: {f1_score(y_val, (oof_preds[val_idx] > 0.5).astype(int)):.5f}")

# ✅ Threshold 최적화
thresholds = np.arange(0.1, 0.9, 0.01)
f1s = [f1_score(y, oof_preds > t) for t in thresholds]
best_thresh = thresholds[np.argmax(f1s)]
print(f"Best Threshold: {best_thresh:.2f}, Best F1 Score: {max(f1s):.5f}")

# ✅ 최종 예측
submission["Cancer"] = (test_preds > best_thresh).astype(int)
submission.to_csv("submission_stacking_kfold_ensemble.csv", index=False)


/usr/local/lib/python3.11/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fold 1 F1: 0.43426
[LightGBM] [Info] Number of positive: 8367, number of negative: 61360
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 765
[LightGBM] [Info] Number of data points in the train set: 69727, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.119997 -> initscore=-1.992463
[LightGBM] [Info] Start training from score -1.992463
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Fold 2 F1: 0.41175


In [ ]:
from google.colab import files
files.download("submission_stacking_kfold_ensemble.csv")

In [ ]:
# ✅ 1. LightGBM 단독 모델에 다른 모델 보조 적용 전략
# 🔸 전략 A: Soft Voting Ensemble
# LightGBM + CatBoost + XGBoost 등을 soft voting으로 앙상블

# 장점: 서로 다른 모델의 강점을 결합

# 구현:

# python
# 복사
# 편집
# from sklearn.ensemble import VotingClassifier
# from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
# from lightgbm import LGBMClassifier

# lgbm = LGBMClassifier(**best_params)
# xgb = XGBClassifier(eval_metric='logloss')
# cat = CatBoostClassifier(verbose=0)

# ensemble = VotingClassifier(
#     estimators=[("lgbm", lgbm), ("xgb", xgb), ("cat", cat)],
#     voting="soft"
# )
# 🔸 전략 B: StackingClassifier
# Level-1 모델: LightGBM, CatBoost, XGBoost

# Meta 모델: Logistic Regression 또는 LGBM

# 장점: 모델 간 의사결정 차이를 메타 모델이 보완

# 구현 예시:

# python
# 복사
# 편집
# from sklearn.ensemble import StackingClassifier
# from sklearn.linear_model import LogisticRegression

# base_models = [
#     ('lgbm', LGBMClassifier(**best_params)),
#     ('xgb', XGBClassifier(eval_metric='logloss')),
#     ('cat', CatBoostClassifier(verbose=0))
# ]
# stack_model = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())
# ✅ 2. LightGBM + SHAP 기반 변수 → 다른 모델 활용
# SHAP으로 파생된 상호작용 feature를 LightGBM에서 먼저 분석

# 이 feature만 추출해서 XGBoost나 CatBoost에 적용

# LightGBM → Feature Engineering, XGB/Cat → 최종 분류기

# 장점: 해석력 + 성능 조화

# ✅ 3. LightGBM + Neural Network 하이브리드 (고급 전략)
# LightGBM의 출력(leaf index 또는 예측값)을 Neural Network의 feature로 사용

# 또는 TabNet 같은 모델과 병렬적으로 조합

# 복잡하지만 성능 상승 여지 있음

# 🔍 추천 실험 순서
# LightGBM 단독 기준 성능 확보 → 완료됨 ✅

# Soft Voting (LGBM + XGB + CatBoost) → 빠른 성능 확인 가능

# StackingClassifier 적용 → 성능 개선 여지 있음

# Focal Loss + Sample Weight 조합 → 미세 튜닝

# SHAP 기반 변수만 추출 후 각 모델 실험

# 딥러닝/TabNet 병합 (필요시)

In [ ]:
# 1. Threshold 최적화 + Class Weight Ensembling
# 서로 다른 threshold를 사용하는 모델 앙상블로 극대화 가능.

# 예: LightGBM(0.40), CatBoost(0.46), XGBoost(0.48) 각각의 최적 threshold로 예측 후 soft voting.

# 📌 실행 예시:

# python
# 복사
# 편집
# final_pred = (0.3 * (lgb_pred > 0.40) +
#               0.3 * (cat_pred > 0.46) +
#               0.4 * (xgb_pred > 0.48)) >= 0.5
# 2. Feature Interaction SHAP Top-N 조합 + LGBM
# 성능이 좋았던 Age × Nodule_Size, T3 × T4, Iodine × Weight Risk 등을 최소 3~5개로 다시 조합

# Interaction term을 직접 곱해서 Feature에 추가 (X['Age*Nodule'] = X['Age'] * X['Nodule_Size'])

# 📌 성능 향상 기대치는 기존보다 약 0.02~0.04

# 3. Target Encoding + Stratified K-Fold로 안정화
# Race, Country, Gender 등 범주형 변수에 Target Encoding (CV 누수 방지)

# 예측력 있는 범주형 feature의 정보 손실 최소화

# python
# 복사
# 편집
# from category_encoders import TargetEncoder

# encoder = TargetEncoder()
# X_cat_encoded = encoder.fit_transform(X[['Race', 'Country']], y)
# 4. StackingClassifier + TTA (Test Time Feature Augmentation)
# 현재 최고의 stacking 결과를 기반으로, test 시 Age, Weight, Nodule_Size 등을 0.95~1.05 비율로 TTA 적용

# Soft voting 결과를 평균하여 예측

# 📌 적용 예시:

# python
# 복사
# 편집
# for scale in [0.95, 1.0, 1.05]:
#     X_test_scaled = X_test.copy()
#     X_test_scaled["Age"] *= scale
#     preds.append(model.predict_proba(X_test_scaled)[:, 1])
# final_preds = np.mean(preds, axis=0)
# 5. 모델 앙상블 구조 개선: LightGBM 메타 + CatBoost/XGB 보조
# 지금의 Stacking도 좋지만, 메타 모델을 LightGBM으로 고정하고,

# 보조 모델들은 다양화 (CatBoost는 논리적 분류에, XGB는 큰 값에 민감)

# python
# 복사
# 편집
# # StackingClassifier에서 final_estimator를 LGBMClassifier로 유지
# # base 모델을 cat + xgb + tuned lgb로 구성

In [ ]:
# 🔝 [현재 성능 기반 향상 가능성 높은 전략 Top 3]
# ✅ 1. Stacking 구조 개선 + Feature Interaction 파생변수 추가 (강력 추천 ⭐)
# 지금 Stacking 구조가 잘 작동 중이므로, 여기에 상호작용 파생변수를 더해 메타모델이 더 똑똑한 판단을 내릴 수 있게 만드는 것이 가장 안전한 강화 전략입니다.

# 방법:

# 기존 stacking 입력 feature에 Age_x_Nodule_Size, T3_Result_x_T4_Result 등 SHAP 기반 상위 상호작용 3~5개 추가.

# 메타모델 입력에도 동일하게 파생 feature를 포함시킴.

# 💡 이미 submission_lgbm_shap_interaction.csv가 괜찮았기 때문에 이 구조와 stacking을 결합시키는 것이 가장 강력한 전략입니다.

# ✅ 2. Class Weight Ensembling (Soft voting에 가중치 적용) (보수적 전략)
# 현재는 StackingClassifier로 진행 중인데, LightGBM, XGBoost, CatBoost 등의 앙상블 가중치를 soft voting 기준으로 최적화하면 성능이 안정적으로 증가할 수 있습니다.

# 예시:

# python
# 복사
# 편집
# voting_model = VotingClassifier(
#     estimators=[
#         ('lgbm', LGBMClassifier(**lgbm_params)),
#         ('xgb', XGBClassifier(**xgb_params)),
#         ('cat', CatBoostClassifier(**cat_params))
#     ],
#     voting='soft',
#     weights=[3, 2, 1]  # 성능 비례 가중치
# )
# Soft voting 기반이지만 Stacking과 조합도 가능합니다 (voting output → meta model input).

# ✅ 3. Test-Time Augmentation (TTA) 적용 (예측 안정화 전략)
# TTA는 tabular data에선 feature dropout / noise 추가 등을 통해 예측 분산을 줄이고 안정적인 결과를 만듭니다.

# 예시: Test data에 noise를 3~5회 추가한 후 예측값 평균

# 특히 logit 기반 soft voting 결과에 TTA를 적용하면 0.01~0.02 F1 개선 효과 기대됩니다.